# Load wordle wordlist

In [2]:
# Load both allowed and answers txt
allowed_path = 'allowed.txt'
answers_path = 'answers.txt'

with open(allowed_path, 'r') as f:
    allowed = list(f.read().splitlines())

with open(answers_path, 'r') as f:
    answers = list(f.read().splitlines())

In [3]:
print(len(allowed))
print(allowed[:10])

10657
['aahed', 'aalii', 'aargh', 'aarti', 'abaca', 'abaci', 'abacs', 'abaft', 'abaka', 'abamp']


In [4]:
print(len(answers))
print(answers[:10])

2315
['aback', 'abase', 'abate', 'abbey', 'abbot', 'abhor', 'abide', 'abled', 'abode', 'abort']


# Find the best starting word
# Link: https://www.youtube.com/watch?v=v68zYyaEmEA

In [33]:
import math
from collections import Counter, defaultdict
import random
from itertools import product

# Load both allowed and answers txt
allowed_path = 'allowed.txt'
answers_path = 'answers.txt'

with open(allowed_path, 'r') as f:
    allowed = list(f.read().splitlines())

with open(answers_path, 'r') as f:
    answers = list(f.read().splitlines())


WORD_LENGTH = 5
GUESSES = 6

GREEN_STATE = 0
YELLOW_STATE = 1
GRAY_STATE = 2


def validate_guess(guess, answer):
    # Using wordle rules, validate a guess against the answer
    guess = guess.lower()
    answer = answer.lower()

    # Initialize all as grey at first
    pattern = [GRAY_STATE] * WORD_LENGTH

    # First pass: check for GREEN matches, exact matches in both position and letter
    # Record wrong letters for pass 2
    leftovers = {}
    for idx in range(WORD_LENGTH):
        if guess[idx] == answer[idx]:
            # Correct letter in the correct position
            # Mark as GREEN_STATE
            pattern[idx] = GREEN_STATE
        else:
            # Wrong position or letter
            # See correct answer letter for this position
            # Store the letter in leftovers for later
            leftovers[answer[idx]] = leftovers.get(answer[idx], 0) + 1

    # Second pass: Check for yellow matches, correct letter but wrong position
    for idx in range(WORD_LENGTH):
        # Only consider wrong, non-green letters
        if pattern[idx] == GRAY_STATE:
            # If the letter exists in the leftovers, aka answer, mark it as YELLOW_STATE
            # Get letter of guess at this index
            letter = guess[idx]

            # If letter exists in leftovers and has a count > 0
            if leftovers.get(letter, 0) > 0:
                # Mark as YELLOW_STATE
                pattern[idx] = YELLOW_STATE
                # Decrease the count of that letter in leftovers
                # This make sures that edge cases with 2 or more of the same letter are handled
                leftovers[letter] -= 1

    return tuple(pattern)

# Build a 1 level tree of patterns based on each word in allowed validating against all answers
# Complexity: O(allowed * Answers)
# Will return a nested dict containing key = allowed word, value = dict of patterns as keys, and a set of answers as values
def build_pattern_dict(allowed, answers):
    # Create a dict to contain all possible answer patterns for each allowed word
    pattern_dict = { word: defaultdict(set) for word in allowed }
    for word in allowed:
        for ans in answers:
            pattern = validate_guess(word, ans)
            pattern_dict[word][pattern].add(ans)
    return pattern_dict

# Preprocess the pattern dict
pattern_dict = build_pattern_dict(allowed, answers)


# Calculate Shannon entropy for a list of counts
def calculate_shannon_entropy(counts):
    # Formula: I = Log2(1/p)
    # I = -Log2(p)
    total = sum(counts)
    return -sum((c/total) * math.log(c/total) for c in counts if c > 0)


# From the calculated pattern dict, pick the best guess based on entropy
def pick_best_guess(remaining, all_patterns):
    # If remaining only left with one word, return it
    if len(remaining) == 1:
        return next(iter(remaining))

    # Var to store the best guess and its score
    best_g = None
    best_score = -1

    # Loop all allowed words
    for g in allowed:
        # Build the 243‐entry vector of "how many remaining answers yield each pattern"
        counts = [len(pattern_dict[g][p] & remaining) for p in all_patterns]
        score = calculate_shannon_entropy(counts)
        if score > best_score:
            best_score, best_g = score, g
    return best_g


def solve_wordle():
    # Randomly select a secret word from the answers
    secret = random.choice(answers)
    print(f"Secret word is: {secret}")

    # Generate all possible patterns
    all_patterns = list(product((0,1,2), repeat=WORD_LENGTH))

    # Init the set of possible answers
    remaining = set(answers)

    # Loop for a maximum number of turns, same as wordle game
    for turn_num in range(0, GUESSES):
        # Find the guess with the highest entropy
        guess = pick_best_guess(remaining, all_patterns)

        # Get the Wordle feedback for that guess
        feedback_pattern = validate_guess(guess, secret)

        # Print turn information
        print(f"Turn {turn_num + 1}: guess = {guess!r} feedback = {feedback_pattern}")

        # Check if the guess is correct
        if guess == secret:
            print(f"Solved in {turn_num + 1}, the secret word was {secret!r}")
            return None
        else:
            # Not correct, prune the remaining answers based on the feedback
            # Find the intersection of the remaining answers and the pattern for this guess
            remaining = remaining.intersection(pattern_dict[guess][feedback_pattern])

    if remaining:
        print(f"Failed to solve within {GUESSES} guesses. Remaining candidates: {remaining}")

In [34]:
solve_wordle()

Secret word is: ozone
Turn 0: guess = 'soare' feedback = (2, 1, 2, 2, 0)
Turn 1: guess = 'bling' feedback = (2, 2, 2, 0, 2)
Turn 2: guess = 'aahed' feedback = (2, 2, 2, 1, 2)
Turn 3: guess = 'ozone' feedback = (0, 0, 0, 0, 0)
Solved in 4, the secret word was 'ozone'


In [ ]:
# Edge cases
print(validate_guess("Speed", "Abide"))
print(validate_guess("Speed", "Erase"))
print(validate_guess("Speed", "Steal"))
print(validate_guess("Speed", "Crepe"))